**Test / Example note book of the functions executed in card_analyzer.py**

### Load example dataset

In [1]:
import os
import sys
import json
import re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import ipywidgets as widgets
from IPython.display import display
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [35]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))
from utils import *
from card_analyzer import *

In [37]:
# Set path of the folder containing dataset
dataset_FolderPath = Path.cwd().parent / 'data' # @dev TBC before each use

# Set path of the File
dataset_FileName = 'AllPrintings.json'
dataset_FilePath = dataset_FolderPath / dataset_FileName

In [52]:
# Load all datasets
data = pd.read_json(dataset_FilePath)
allSets = data.iloc[2:]['data'] # 2 first rows of JSON files are metadata

In [54]:
# Load a test dataset
set_code = 'OTJ'
cards = loadSet(allSets, set_code, restriction='baseSet')
cards

name               keywords  manaValue      manaCost  \
0          Another Round                    NaN        3.0  {X}{X}{2}{W}   
1    Archangel of Tithes               [Flying]        4.0  {1}{W}{W}{W}   
2      Armored Armadillo                 [Ward]        1.0           {W}   
3       Aven Interrupter  [Flash, Flying, Plot]        3.0     {1}{W}{W}   
4       Bounding Felidar               [Saddle]        6.0        {5}{W}   
..                   ...                    ...        ...           ...   
266    Botanical Sanctum                    NaN        0.0           NaN   
267  Concealed Courtyard                    NaN        0.0           NaN   
268    Inspiring Vantage                    NaN        0.0           NaN   
269     Spirebluff Canal                    NaN        0.0           NaN   
270      Jace Reawakened                 [Plot]        2.0        {U}{U}   

    colorIdentity  power  toughness    rarity           types  \
0             [W]    NaN        NaN      rare       [Sorcery]   
1             [W]    3.0        5.0    mythic      [Creature]   
2             [W]    0.0        4.0    common      [Creature]   
3             [W]    2.0        2.0      rare      [Creature]   
4             [W]    4.0        7.0  uncommon      [Creature]   
..            ...    ...        ...       ...             ...   
266        [G, U]    NaN        NaN      rare          [Land]   
267        [B, W]    NaN        NaN      rare          [Land]   
268        [R, W]    NaN        NaN      rare          [Land]   
269        [R, U]    NaN        NaN      rare          [Land]   
270           [U]    NaN        NaN    mythic  [Planeswalker]   

                                                  text  
0    Exile any number of creatures you control, the...  
1    Flying\nAs long as Archangel of Tithes is unta...  
2    Ward {1} (Whenever this creature becomes the t...  
3    Flash\nFlying\nWhen Aven Interrupter enters, e...  
4    Whenever Bounding Felidar attacks while saddle...  
..                                                 ...  
266  Botanical Sanctum enters tapped unless you con...  
267  Concealed Courtyard enters tapped unless you c...  
268  Inspiring Vantage enters tapped unless you con...  
269  Spirebluff Canal enters tapped unless you cont...  
270  You can't cast Jace Reawakened during your fir...  

[271 rows x 10 columns]

In [279]:
# TEST 1: Apply filters

def isType(s, typelist):
    return any(t in s for t in typelist)

def isPermanent(s):
    return isType(s, ['Land', 'Creature', 'Artifact', 'Enchantment', 'Planeswalker', 'Battle'])

def createsToken(s):
    pattern = r'creat(e|es) .*? creature token'
    return bool(re.search(pattern, s, re.IGNORECASE | re.DOTALL))

def isETB(s):
    pattern = r'when .*? ente(r|rs)'
    return bool(re.search(pattern, s, re.IGNORECASE | re.DOTALL))

# Test
c = cards.loc[58]
c.text
"""
Outlaw Stitcher

"When Outlaw Stitcher enters, create a 2/2 blue and black Zombie Rogue creature token, then put two +1/+1 counters 
on that token for each spell you've cast this turn other than the first.\nPlot {4}{U} (You may pay {4}{U} and exile 
this card from your hand. Cast it as a sorcery on a later turn without paying its mana cost. Plot only as a sorcery.)"
"""

print(isType(c.types,['Creature']))
print(isPermanent(c.types))
print(createsToken(c.text))
print(isETB(c.text))

# all permanents that creates token on ETB
a = cards['types'].apply(isPermanent)
b = cards['text'].apply(createsToken)
c = cards['text'].apply(isETB)

result = cards[a & b & c]
result

True
True
True
True


name keywords  manaValue      manaCost  \
17          Lassoed by the Law      NaN        4.0        {3}{W}   
24           Prosperity Tycoon      NaN        4.0        {3}{W}   
58             Outlaw Stitcher   [Plot]        4.0        {3}{U}   
98                 Rakish Crew      NaN        3.0        {2}{B}   
101              Rictus Robber   [Plot]        4.0        {3}{B}   
127        Hellspur Posse Boss      NaN        4.0     {2}{R}{R}   
136               Prickly Pair      NaN        3.0        {2}{R}   
195    Bonny Pall, Clearcutter  [Reach]        6.0  {3}{G}{U}{U}   
202  Ertha Jo, Frontier Mentor      NaN        4.0     {2}{R}{W}   

    colorIdentity  power  toughness    rarity          types  \
17            [W]    NaN        NaN  uncommon  [Enchantment]   
24            [W]    4.0        2.0  uncommon     [Creature]   
58            [U]    1.0        4.0  uncommon     [Creature]   
98            [B]    NaN        NaN  uncommon  [Enchantment]   
101           [B]    4.0        3.0  uncommon     [Creature]   
127           [R]    2.0        4.0      rare     [Creature]   
136           [R]    2.0        2.0    common     [Creature]   
195        [G, U]    6.0        5.0      rare     [Creature]   
202        [R, W]    2.0        4.0  uncommon     [Creature]   

                                                  text  
17   When Lassoed by the Law enters, exile target n...  
24   When Prosperity Tycoon enters, create a 1/1 re...  
58   When Outlaw Stitcher enters, create a 2/2 blue...  
98   When Rakish Crew enters, create a 1/1 red Merc...  
101  When Rictus Robber enters, if a creature died ...  
127  Other outlaws you control have haste. (Assassi...  
136  When Prickly Pair enters, create a 1/1 red Mer...  
195  Reach\nWhen Bonny Pall, Clearcutter enters, cr...  
202  When Ertha Jo, Frontier Mentor enters, create ...